In [27]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [28]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\data_values.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\indicator_type.xlsx',sheet_name='Sheet1')

In [29]:
#defining normalization function for (dam capicity)/(surface rwr)
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(ser1.max()-ser1[i])/(ser1.max()-ser2[i])
        else:
            result[i]=(ser1[i])/(ser2[i])
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation functions
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result

#defining weighted aggregation functions
def weighted_addminagg(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    dfmin=dfi.min(axis=1)
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result=a*dfmin+(1-a)*weighted_sum
    return result

def weighted_sum(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    return weighted_sum

def geomean(dfi):
    size=len(dfi.columns)
    geomean=(dfi.product(axis=1)**(1/size))
    return geomean

def weighted_geomean(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_addgeo(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result= a*weighted_geomean+(1-a)*weighted_sum
    return result

In [30]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [53]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/17)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr
variables[['province','withdraw_agr']]

,province,withdraw_agr
0,azsh,2331.9400
1,azgh,3950.0850
2,arda,1536.9000
3,esfe,5914.0707
4,albo,790.3840
5,ilam,893.7994
6,bush,929.6703
7,tehr,2440.5500
8,chba,1728.4979
9,khoj,1321.2820


In [50]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['irwr_percap']=(variables.irwr*(10**6)/data.population)/(1+data['pop_growth']/100)
#Dimension 2: access

sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['under_stress_pop']=data['under_stress_pop']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr

#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind

sub_indicators.index=variables['province']
sub_indicators.loc['khuz','damcap_rwr_ratio']=1
# sub_indicators['damcap_rwr_ratio']
sub_indicators['efficiency_ind']

province
azsh      87.289034
azgh      22.519221
arda      41.620506
esfe      78.001811
albo      37.255510
ilam       3.570169
bush     195.044482
tehr     148.856493
chba       7.824678
khoj       7.988948
khor      23.562544
khosh     29.589705
khuz      14.713898
zanj      14.755878
semn      43.590806
sisba     10.433575
fars      31.386879
qazv      60.384361
qom       80.206649
kord       5.781043
kerm      19.230663
kersh     27.725796
kohbu      9.946154
gole      34.034579
gilan     11.980737
lores     26.584001
mazan     74.926254
marka    144.051497
hormo    140.979562
hame      30.898499
yazd      49.166008
Name: efficiency_ind, dtype: float64

In [33]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)    

In [34]:

variation_coeff_agg=pd.Series(sub_indicators[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])


In [42]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(sub_indicators[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=1

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
gw_deficit_bins=pd.cut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
gw_modifier=pd.Series(index=variables['province'],dtype='float64')
gw_modifier[gw_deficit_bins=='Q1']=0.7
gw_modifier[gw_deficit_bins=='Q2']=0.8
gw_modifier[gw_deficit_bins=='Q3']=0.9
gw_modifier[gw_deficit_bins=='Q4']=1
normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)

for col in normalized.columns:
    normalized.loc[normalized[col]<0.01,col]=0.01
normalized['damcap_rwr_ratio']
ideal_damcap

province
azsh     1.0
azgh     1.0
arda     1.0
esfe     0.9
albo     0.9
ilam     0.8
bush     0.7
tehr     0.9
chba     0.9
khoj     0.8
khor     0.9
khosh    0.9
khuz     0.8
zanj     0.9
semn     0.9
sisba    0.7
fars     0.8
qazv     0.9
qom      0.9
kord     0.9
kerm     0.7
kersh    0.9
kohbu    0.8
gole     1.0
gilan    1.0
lores    0.9
mazan    1.0
marka    0.9
hormo    0.7
hame     0.9
yazd     0.8
dtype: float64

In [49]:
res_sub_columns=['ACV','MCV','Dag','Dan','GWDagr','GWDind','GWDurb','GWDrur','SWS','GWS','APA','ATA','IRWR']
acc_sub_columns=['WAurb','WArur','USP','SAurb','SArur','TPC','WQ','DC']
eco_sub_columns=['AE','MI','EL','NRWurb','NRWrur','IE']
sub_columns1_list= ['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R7']+['A1','A1','A1','A2','A2','A2','A3','A3']+['E1','E1','E1','E2','E2','E3']
sub_columns2_list=res_sub_columns+acc_sub_columns+eco_sub_columns
arraycolumns=[sub_columns1_list]+[sub_columns2_list]
normalized_pp=pd.DataFrame(normalized).round(decimals=4)
normalized_pp.columns=sub_columns2_list
normalized_pp.loc[:,['AE','MI','EL','NRWurb','NRWrur','IE']]

,AE,MI,EL,NRWurb,NRWrur,IE
province,,,,,,
azsh,0.6485,0.1898,1.0000,0.7040,0.478,0.4220
azgh,0.3468,0.1606,0.6779,0.5820,0.634,0.0898
arda,0.5183,0.0671,1.0000,0.4867,0.612,0.1878
esfe,0.1655,0.1125,0.7487,0.7953,0.608,0.3744
albo,0.7073,0.0957,0.9144,0.5900,0.282,0.1654
ilam,0.2544,0.3726,1.0000,0.6090,0.596,0.0100
bush,0.5424,0.2888,1.0000,0.4617,0.506,0.9746
tehr,0.5123,0.1032,0.9448,0.5353,0.450,0.7377
chba,0.2417,0.5510,1.0000,0.5420,0.292,0.0145


In [11]:
# normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
# normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
# normalized_lvl2[('R5','CV')]=geomean(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
# normalized_lvl2[('R6','GD')]=weighted_sum(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
# normalized_lvl2[('R6','GWD')]=weighted_sum(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)
# normalized_lvl2.head()

In [48]:
## Assessing water security using ageomtric mean aggregation rules

#  calculating indicators 
indicators_pp=pd.DataFrame(index=sub_indicators.index)
temp=pd.DataFrame(index=sub_indicators.index)
temp['GD']=geomean(normalized_pp[['Dag','Dan']])
temp['GWD']=geomean(normalized_pp[['GWDagr','GWDind','GWDurb','GWDrur']])
# indicators_pp.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_pp['IRWR']=normalized_pp['IRWR']
indicators_pp['CC']=geomean(normalized_pp[['ACV','MCV','APA','ATA']]).round(decimals=4)
indicators_pp['WS']=geomean(normalized_pp[['SWS','GWS']]).round(decimals=4)
# if a province has low groundwater reservoir deficit then high dependency on groundwater is considered okay i.e. if GD> 0.8 then GWD not involved in aggregation
indicators_pp.loc[temp['GD']<0.8,'GW']=geomean(temp[['GD','GWD']]).round(decimals=4)
indicators_pp.loc[temp['GD']>0.8,'GW']=temp['GD']
indicators_pp['DW']=geomean(normalized_pp[['WAurb','WArur','USP']]).round(decimals=4)
indicators_pp['WQ']=normalized_pp['WQ']
indicators_pp['DC']=normalized_pp['DC']
indicators_pp['SS']=geomean(normalized_pp[['SAurb','SArur','TPC']]).round(decimals=4)
indicators_pp['Eagr']=geomean(normalized_pp[['AE','MI','EL']]).round(decimals=4)
indicators_pp['IE']=normalized_pp['IE']
indicators_pp['NRW']=geomean(normalized_pp[['NRWurb','NRWrur']]).round(decimals=4)
indicators_pp.loc[:,['NRW','IE','Eagr']]

,NRW,IE,Eagr
province,,,
azsh,0.5801,0.4220,0.4974
azgh,0.6074,0.0898,0.3355
arda,0.5458,0.1878,0.3503
esfe,0.6954,0.3744,0.3052
albo,0.4079,0.1654,0.5017
ilam,0.6025,0.0100,0.4560
bush,0.4833,0.9746,0.5783
tehr,0.4908,0.7377,0.4671
chba,0.3978,0.0145,0.5107


In [45]:
# #  calculating dimensions
Dimensions=pd.DataFrame(index=sub_indicators.index)
Dimensions['Resource']=geomean(indicators_pp[['IRWR','CC','WS','GW']]).round(decimals=4)
Dimensions['Access']=geomean(indicators_pp[['DW','DC','SS']]).round(decimals=4)
Dimensions['Economy']=geomean(indicators_pp[['Eagr','IE','NRW']]).round(decimals=4)

# #  calculating WSI
Dimensions.loc[['azsh','albo','tehr','kohbu','lores','mazan'],:].transpose()

province,azsh,albo,tehr,kohbu,lores,mazan
Resource,0.3087,0.0896,0.0490,0.4969,0.8115,0.8101
Access,0.3402,0.4054,0.5093,0.2524,0.0989,0.1364
Economy,0.4956,0.3235,0.5530,0.1446,0.3236,0.4630


In [46]:
## Assessing water security using geomtric mean aggregation rules

#  calculating indicators 
# indicators_pp=pd.DataFrame(index=sub_indicators.index)
# temp=pd.DataFrame(index=sub_indicators.index)
# temp['GD']= normalized_pp[['Dag','Dan']].mean(axis=1)
# temp['GWD']= normalized_pp[['GWDagr','GWDind','GWDurb','GWDrur']].mean(axis=1)
# # indicators_pp.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
# indicators_pp['IRWR']=normalized_pp['IRWR']
# indicators_pp['CC']= normalized_pp[['ACV','MCV','APA','ATA']].mean(axis=1).round(decimals=4)
# indicators_pp['WS']= normalized_pp[['SWS','GWS']].mean(axis=1).round(decimals=4)
# # if a province has low groundwater reservoir deficit then high dependency on groundwater is considered okay i.e. if GD> 0.8 then GWD not involved in aggregation
# indicators_pp.loc[temp['GD']<0.8,'GW']= temp[['GD','GWD']].mean(axis=1).round(decimals=4)
# indicators_pp.loc[temp['GD']>0.8,'GW']=temp['GD']
# indicators_pp['DW']= normalized_pp[['WAurb','WArur','USP']].mean(axis=1).round(decimals=4)
# indicators_pp['WQ']=normalized_pp['WQ']
# indicators_pp['DC']=normalized_pp['DC']
# indicators_pp['SS']= normalized_pp[['SAurb','SArur','TPC']].mean(axis=1).round(decimals=4)
# indicators_pp['Eagr']= normalized_pp[['AE','MI','EL']].mean(axis=1).round(decimals=4)
# indicators_pp['IE']=normalized_pp['IE']
# indicators_pp['NRW']= normalized_pp[['NRWurb','NRWrur']].mean(axis=1).round(decimals=4)

# # #  calculating dimensions
# Dimensions_arith=pd.DataFrame(index=sub_indicators.index)
# Dimensions_arith['Resource']= geomean(indicators_pp[['IRWR','CC','WS','GW']]).round(decimals=4)
# Dimensions_arith['Access']= geomean(indicators_pp[['DW','WQ','DC','SS']]).round(decimals=4)
# Dimensions_arith['Economy']= geomean(indicators_pp[['Eagr','IE','NRW']]).round(decimals=4)

# Dimensions_arith['Resource_arith']= indicators_pp[['IRWR','CC','WS','GW']].mean(axis=1).round(decimals=4)
# Dimensions_arith['Access_arith']= indicators_pp[['DW','WQ','DC','SS']].mean(axis=1).round(decimals=4)
# Dimensions_arith['Economy_arith']= indicators_pp[['Eagr','IE','NRW']].mean(axis=1).round(decimals=4)
# # #  calculating WSI
# Dimensions_arith['WSI']=geomean( Dimensions[['Resource','Access','Economy']]).round(decimals=4)
# Dimensions_arith['WSI_arith']=Dimensions[['Resource_arith','Access_arith','Economy_arith']].mean(axis=1).round(decimals=4)
# Dimensions_arith.sort_values('WSI',ascending=False)

